In [ ]:
import numpy as np        #넘파이 라이브러리 불러오기
import pandas as pd        #판다스 라이브러리 불러오기

In [ ]:
# csv 파일 불러오기
df = pd.read_csv("C:/Users/mi38m/OneDrive/바탕 화면/미영 분석/주차장/raw data/자치구 단위 서울 생활인구(내국인).csv", encoding = 'cp949')

# 데이터프레임 확인
df

In [ ]:
# 열 확인
df.columns

In [ ]:
# 기준일 2024년 3월 16일 ~ 3월 22일까지 1주치 샘플 데이터, 20대 이상 생활인구수 계산을 위해 필요한 열만 출력
filtered_df = df[(df['기준일ID'] >= 20240316) & (df['기준일ID'] <= 20240322)][['기준일ID', '자치구코드', '총생활인구수', '남자0세부터9세생활인구수', '남자10세부터14세생활인구수', '남자15세부터19세생활인구수',
                                           '여자0세부터9세생활인구수', '여자10세부터14세생활인구수', '여자15세부터19세생활인구수']]

In [ ]:
# 총 생활인구수에서 0~19세 인구수를 제외한 (운전 가능한) 20대 이상 생활인구수 열 생성
filtered_df['20대이상 생활인구수'] = filtered_df['총생활인구수'] - filtered_df['남자0세부터9세생활인구수'] - filtered_df['남자10세부터14세생활인구수'] - filtered_df['남자15세부터19세생활인구수']
- filtered_df['여자0세부터9세생활인구수'] - filtered_df['여자10세부터14세생활인구수'] - filtered_df['여자15세부터19세생활인구수']

In [ ]:
# 불필요한 열 삭제
filtered_df.drop(['총생활인구수', '남자0세부터9세생활인구수', '남자10세부터14세생활인구수', '남자15세부터19세생활인구수',
                                           '여자0세부터9세생활인구수', '여자10세부터14세생활인구수', '여자15세부터19세생활인구수'], axis=1, inplace=True)

In [ ]:
# 20대이상 생활인구수를 정수로 변환
filtered_df['20대이상 생활인구수'] = filtered_df['20대이상 생활인구수'].astype(int)

In [ ]:
# 자치구코드별 20대이상 생활인구수 합계 구하기
grouped_df = filtered_df.groupby('자치구코드')['20대이상 생활인구수'].sum().reset_index()  # .reset_index()를 통해 데이터프레임으로

In [ ]:
# 20대이상 일평균 생활인구수 계산
grouped_df['20대이상_일평균 생활인구수'] = grouped_df['20대이상 생활인구수'] / 7

In [ ]:
# 20대이상 일평균 생활인구수를 정수로 변환
grouped_df['20대이상_일평균 생활인구수'] = grouped_df['20대이상_일평균 생활인구수'].astype(int)

In [ ]:
# 자치구코드를 행정구역명으로 변경하기 위한 딕셔너리
sigungu_change = dict(zip([11110, 11140, 11170, 11200, 11215, 11230, 11260, 11290,
                           11305, 11320, 11350, 11380, 11410, 11440, 11470, 11500,
                           11530, 11545, 11560, 11590, 11620, 11650, 11680, 11710, 11740],
                          ['종로구', '중구', '용산구', '성동구', '광진구', '동대문구', '중랑구', '성북구',
                           '강북구', '도봉구', '노원구', '은평구', '서대문구', '마포구', '양천구', '강서구',
                           '구로구', '금천구', '영등포구', '동작구', '관악구', '서초구', '강남구', '송파구', '강동구']))

In [ ]:
# 자치구코드를 행정구역명으로 변경
grouped_df['자치구코드'] = grouped_df['자치구코드'].replace(sigungu_change)

In [ ]:
# 열 이름 변경
grouped_df.rename(columns={'20대이상 생활인구수': '20대이상_생활인구수_1주합계'}, inplace=True)

In [ ]:
grouped_df.rename(columns={'20대이상_일평균 생활인구수': '20대이상_생활인구수_하루평균'}, inplace=True)

In [ ]:
# 자치구코드(행정구역명)를 행으로 사용
grouped_df.set_index('자치구코드', inplace=True)

In [ ]:
# 행 이름을 행정구역으로
grouped_df.index.name = '행정구역'

In [ ]:
# 행(행정구역명) 기준으로 가나다순 정렬
grouped_df = grouped_df.sort_index()

In [ ]:
#새로운 csv로 저장하기
grouped_df.to_csv("C:/Users/mi38m/OneDrive/바탕 화면/미영 분석/주차장/전처리완료/생활인구_1주샘플_전처리", index=True, encoding = 'cp949')

#index = True 는 인덱스도 저장하는 옵션, False는 인덱스는 저장하지 않도록 하는 옵션
#데이터 프레임에 한글이 있을 경우, 인코딩으로 해줘야함. utf-8이나, cp949 중에서 utf8는 오류가 발생함